In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 31.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


# Test

In [ ]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data['vehicle_capacities'] = [15, 15, 15, 15]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 6208
Route for vehicle 0:
 0 Load(0) ->  4 Load(4) ->  3 Load(6) ->  1 Load(7) ->  7 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) ->  14 Load(4) ->  16 Load(12) ->  10 Load(14) ->  9 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) ->  12 Load(2) ->  11 Load(3) ->  15 Load(11) ->  13 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) ->  8 Load(8) ->  2 Load(9) ->  6 Load(13) ->  5 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Total distance of all routes: 6208m
Total load of all routes: 60


# OR-Tools


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import time

In [ ]:
# Load the XML file
tree = ET.parse('CMT06.xml')
root = tree.getroot()

In [ ]:
# Extract the node and edge data from the XML file
node_list = []
demand_list = []
for node in root.findall("./network/nodes/"):
    node_id = int(node.attrib['id'])
    node_type = int(node.attrib['type'])
    node_x = float(node.find('cx').text)
    node_y = float(node.find('cy').text)
    node_list.append([node_id, node_type, node_x, node_y])
for demand in root.findall("./requests/"):
    node_id = int(demand.attrib['node'])
    node_demand = float(demand.find('quantity').text)
    demand_list.append([node_id, node_demand])

In [ ]:
# Convert the node_list and edge_list into dataframes
node_df = pd.DataFrame(node_list, columns=["id", "type", "x", "y"])
demand_df = pd.DataFrame(demand_list, columns=["id", "demand"])

In [ ]:
from scipy.spatial import distance_matrix

In [ ]:
# Extract the x and y coordinates as a numpy array
coords = node_df[['x', 'y']].to_numpy()

# Calculate the distance matrix using the Euclidean distance metric
dist_matrix = distance_matrix(coords, coords)

In [ ]:
demand_df['demand'].sum()

777.0

In [ ]:
demand = list(demand_df['demand'])

In [ ]:
demand.append(0)

In [ ]:
dist_matrix*1000

array([[    0.        , 12369.31687685, 19209.3727123 , ...,
        26400.75756489, 24207.43687382, 13892.44398945],
       [12369.31687685,     0.        , 15297.05854078, ...,
        21023.79604163, 13892.44398945, 21023.79604163],
       [19209.3727123 , 15297.05854078,     0.        , ...,
        36221.54055255, 27294.68812791, 32557.6411922 ],
       ...,
       [26400.75756489, 21023.79604163, 36221.54055255, ...,
            0.        , 12041.59457879, 21633.30765278],
       [24207.43687382, 13892.44398945, 27294.68812791, ...,
        12041.59457879,     0.        , 26172.5046566 ],
       [13892.44398945, 21023.79604163, 32557.6411922 , ...,
        21633.30765278, 26172.5046566 ,     0.        ]])

In [ ]:
temp_matrix = dist_matrix * 1000
new_dist_matrix = temp_matrix.astype(int)

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = new_dist_matrix
    data['demands'] = [int(x) for x in demand]
    data['vehicle_capacities'] = [160, 160, 160, 160, 160]
    data['num_vehicles'] = 5
    data['depot'] = 50
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue() / 1000}')
    total_distance = 0
    total_load = 0
    routes = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        current_routes = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            current_routes.append(node_index)
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index), route_load)
        current_routes.append(data['depot'])
        routes.append(current_routes)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/1000)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance/1000))
    print('Total load of all routes: {}'.format(total_load))
    print(routes)


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH)
    search_parameters.time_limit.FromSeconds(5)

    # Solve the problem.
    start_time = time.time()
    solution = routing.SolveWithParameters(search_parameters)
    print("--- %s seconds ---" % (time.time() - start_time))
    # Print solution on console.
    # Print the solution.


    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == '__main__':
    main()

--- 5.001806259155273 seconds ---
Objective: 595.054
Route for vehicle 0:
 50 Load(0) ->  26 Load(15) ->  30 Load(26) ->  25 Load(33) ->  42 Load(44) ->  23 Load(54) ->  12 Load(77) ->  40 Load(104) ->  39 Load(111) ->  18 Load(120) ->  41 Load(133) ->  43 Load(149) ->  36 Load(158) ->  50 Load(158)
Distance of the route: 178.454m
Load of the route: 158

Route for vehicle 1:
 50 Load(0) ->  16 Load(3) ->  14 Load(13) ->  44 Load(23) ->  32 Load(46) ->  38 Load(60) ->  9 Load(65) ->  29 Load(84) ->  33 Load(110) ->  15 Load(125) ->  10 Load(144) ->  31 Load(156) ->  50 Load(156)
Distance of the route: 120.67m
Load of the route: 156

Route for vehicle 2:
 50 Load(0) ->  0 Load(7) ->  21 Load(15) ->  2 Load(31) ->  34 Load(48) ->  35 Load(54) ->  27 Load(68) ->  7 Load(91) ->  47 Load(108) ->  6 Load(127) ->  22 Load(143) ->  5 Load(158) ->  50 Load(158)
Distance of the route: 131.594m
Load of the route: 158

Route for vehicle 3:
 50 Load(0) ->  1 Load(30) ->  19 Load(58) ->  28 Load(64) 

#OR-Tools Servicetime


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [ ]:
# Load the XML file
tree = ET.parse('CMT06.xml')
root = tree.getroot()

In [ ]:
# Extract the node and edge data from the XML file
node_list = []
demand_list = []
for node in root.findall("./network/nodes/"):
    node_id = int(node.attrib['id'])
    node_type = int(node.attrib['type'])
    node_x = float(node.find('cx').text)
    node_y = float(node.find('cy').text)
    node_list.append([node_id, node_type, node_x, node_y])
for demand in root.findall("./requests/"):
    node_id = int(demand.attrib['node'])
    node_demand = float(demand.find('quantity').text)
    demand_list.append([node_id, node_demand])

In [ ]:
# Convert the node_list and edge_list into dataframes
node_df = pd.DataFrame(node_list, columns=["id", "type", "x", "y"])
demand_df = pd.DataFrame(demand_list, columns=["id", "demand"])

In [ ]:
from scipy.spatial import distance_matrix

In [ ]:
# Extract the x and y coordinates as a numpy array
coords = node_df[['x', 'y']].to_numpy()

# Calculate the distance matrix using the Euclidean distance metric
dist_matrix = distance_matrix(coords, coords)

In [ ]:
demand_df['demand'].sum()

777.0

In [ ]:
demand = list(demand_df['demand'])

In [ ]:
demand.append(0)

In [ ]:
dist_matrix*1000

array([[    0.        , 12369.31687685, 19209.3727123 , ...,
        26400.75756489, 24207.43687382, 13892.44398945],
       [12369.31687685,     0.        , 15297.05854078, ...,
        21023.79604163, 13892.44398945, 21023.79604163],
       [19209.3727123 , 15297.05854078,     0.        , ...,
        36221.54055255, 27294.68812791, 32557.6411922 ],
       ...,
       [26400.75756489, 21023.79604163, 36221.54055255, ...,
            0.        , 12041.59457879, 21633.30765278],
       [24207.43687382, 13892.44398945, 27294.68812791, ...,
        12041.59457879,     0.        , 26172.5046566 ],
       [13892.44398945, 21023.79604163, 32557.6411922 , ...,
        21633.30765278, 26172.5046566 ,     0.        ]])

In [ ]:
temp_matrix = dist_matrix * 1000
new_dist_matrix = temp_matrix.astype(int)

In [ ]:
new_dist_matrix

array([[    0, 12369, 19209, ..., 26400, 24207, 13892],
       [12369,     0, 15297, ..., 21023, 13892, 21023],
       [19209, 15297,     0, ..., 36221, 27294, 32557],
       ...,
       [26400, 21023, 36221, ...,     0, 12041, 21633],
       [24207, 13892, 27294, ..., 12041,     0, 26172],
       [13892, 21023, 32557, ..., 21633, 26172,     0]])

In [ ]:
demands = [int(x) for x in demand]

In [ ]:
service_time = [10000] * len(demand)

In [ ]:
len(service_time)

51

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import time
import random
random.seed(10)

LIMIT = 200 * 1000
SERVICE_TIME = 10 * 1000

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = new_dist_matrix
    data['demands'] = demands
    data['vehicle_capacities'] = [160, 160, 160, 160, 160, 160, 160, 160, 160, 160]
    #data['limit'] = [LIMIT, LIMIT, LIMIT, LIMIT, LIMIT, LIMIT, LIMIT, LIMIT, LIMIT, LIMIT]
    data['num_vehicles'] = 10
    data['depot'] = 50
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue() / 1000}')
    total_distance = 0
    total_load = 0
    routes = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        route_time = 0
        count = 0
        current_routes = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            current_routes.append(node_index)

            if count > 0:
                temp = data['distance_matrix'][prev_index][node_index] + SERVICE_TIME
                #print(data['distance_matrix'][prev_index][node_index])
                route_time += temp

            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            prev_index = node_index
            count += 1

        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index), route_load)
        current_routes.append(data['depot'])
        routes.append(current_routes)
        #print(current_routes)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/1000)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        plan_output += 'Time of the route: {}\n'.format(route_time/1000)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance/1000))
    print('Total load of all routes: {}'.format(total_load))
    print(routes)




def main():

    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Add Check constraint.
    def check_callback(from_index, to_index):
        """Returns the check value between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = data['distance_matrix'][from_node][to_node]
        if to_node == data['depot']:
            return distance
        else:
            return distance + SERVICE_TIME

    check_callback_index = routing.RegisterTransitCallback(check_callback)

    routing.AddDimension(
        check_callback_index,
        0,  # null slack
        #data['limit'],  # maximum check value
        LIMIT,
        True,  # start cumul to zero
        'Check')



    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH) #GREEDY_DESCENT
    search_parameters.time_limit.FromSeconds(60)

    # Solve the problem.
    start_time = time.time()
    solution = routing.SolveWithParameters(search_parameters)
    print("--- %s seconds ---" % (time.time() - start_time))
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()



--- 60.00017952919006 seconds ---
Objective: 567.955
Route for vehicle 0:
 50 Load(0) ->  26 Load(15) ->  47 Load(32) ->  7 Load(55) ->  25 Load(62) ->  30 Load(73) ->  27 Load(87) ->  21 Load(95) ->  1 Load(125) ->  31 Load(137) ->  50 Load(137)
Distance of the route: 91.485m
Load of the route: 137
Time of the route: 171.485

Route for vehicle 1:
 50 Load(0) ->  16 Load(3) ->  43 Load(19) ->  44 Load(29) ->  32 Load(52) ->  14 Load(62) ->  36 Load(71) ->  11 Load(100) ->  50 Load(100)
Distance of the route: 80.084m
Load of the route: 100
Time of the route: 142.022

Route for vehicle 2:
 50 Load(0) ->  17 Load(41) ->  12 Load(64) ->  40 Load(91) ->  39 Load(98) ->  18 Load(107) ->  41 Load(120) ->  3 Load(129) ->  46 Load(154) ->  50 Load(154)
Distance of the route: 103.15m
Load of the route: 154
Time of the route: 173.717

Route for vehicle 3:
 50 Load(0) ->  50 Load(0)
Distance of the route: 0.0m
Load of the route: 0
Time of the route: 0.0

Route for vehicle 4:
 50 Load(0) ->  13 Loa